In [1]:
import pandas as pd
import torch.nn as nn
import torch
import numpy as np
from talib import abstract

In [2]:
class SelectItem(torch.nn.Module):#這是用來取出多個輸出其中一個的輸出，如果不用sequential的話，就可以不用這個
    def __init__(self, item_index):
        super(SelectItem, self).__init__()
        self._name = 'selectitem'
        self.item_index = item_index

    def forward(self, inputs):
        return inputs[self.item_index]

class crypto_classfier_ver5(nn.Module):#CNN+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver5, self).__init__()
        self.name = "CCV-5"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(12,64),
            torch.nn.Linear(64,128),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.GRU(64, 32, 25),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.GRU(32, 16, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(16,2),
            torch.nn.Softmax(dim=0)
            )
    def forward(self, x):
        x = self.net(x)
        return x

In [3]:
model = torch.load('../model/CCV5.pt',map_location=torch.device('cpu'))
data = pd.read_csv('../data/ADA_USDT_15m.csv')

In [4]:
data

,time,open,high,low,close,volume
0,2022-10-01 00:00:00,0.4346,0.4361,0.4339,0.4361,899989.1
1,2022-10-01 00:15:00,0.4360,0.4364,0.4353,0.4358,994595.6
2,2022-10-01 00:30:00,0.4358,0.4359,0.4329,0.4333,1011557.0
3,2022-10-01 00:45:00,0.4334,0.4342,0.4330,0.4333,409916.9
4,2022-10-01 01:00:00,0.4333,0.4333,0.4325,0.4329,798868.6
...,...,...,...,...,...,...
8924,2023-01-02 11:45:00,0.2537,0.2542,0.2537,0.2540,491283.2
8925,2023-01-02 12:00:00,0.2541,0.2547,0.2541,0.2544,590081.8
8926,2023-01-02 12:15:00,0.2543,0.2547,0.2539,0.2544,485615.1
8927,2023-01-02 12:30:00,0.2543,0.2545,0.2540,0.2540,351837.2


In [5]:
model.eval()

crypto_classfier_ver5(
  (net): Sequential(
    (0): Conv1d(10, 20, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ELU(alpha=1.0)
    (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(20, 40, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ELU(alpha=1.0)
    (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(40, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): Linear(in_features=12, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=128, bias=True)
    (9): GRU(128, 64, num_layers=25)
    (10): SelectItem()
    (11): Dropout(p=0.5, inplace=False)
    (12): GRU(64, 32, num_layers=25)
    (13): SelectItem()
    (14): Dropout(p=0.5, inplace=False)
    (15): GRU(32, 16, num_layers=25)
    (16): SelectItem()
    (17): SelectItem()
    (18): Dropout(p=0.5, inplace=False)
    (19): Linear(in_features=16, out_features=2, bias=True)
    (20): Softmax(dim=0)
  )
)

In [6]:
from sklearn.preprocessing import MinMaxScaler
data['RSI'] = abstract.RSI(data, timeperiod=14)
data['MACD'] = abstract.MACD(data, fastperiod=12, slowperiod=26, signalperiod=9)['macd'] #只取MACD
data['OBV'] = abstract.OBV(data, timeperiod=14)
data['CCI'] = abstract.CCI(data, timeperiod=14)
data['ATR'] = abstract.ATR(data, timeperiod=14)
data['ADX'] = abstract.ADX(data, timeperiod=14)
data['MFI'] = abstract.MFI(data, timeperiod=14)
data['CLOSE_percent'] = data['close'].pct_change()
data['UP'] = data['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
data['DOWN'] = data['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
data['UP'] = data['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
data['DOWN'] = data['DOWN'].shift(-1)

data = data.dropna()

data

,time,open,high,low,close,volume,RSI,MACD,OBV,CCI,ATR,ADX,MFI,CLOSE_percent,UP,DOWN
33,2022-10-01 08:15:00,0.4337,0.4350,0.4334,0.4338,4464874.4,49.942739,-0.000119,3.504787e+06,145.526746,0.001131,15.868327,69.424814,0.000461,0.0,1.0
34,2022-10-01 08:30:00,0.4339,0.4342,0.4328,0.4332,616120.1,45.600201,-0.000113,2.888667e+06,72.832370,0.001150,15.261395,63.556057,-0.001383,0.0,1.0
35,2022-10-01 08:45:00,0.4333,0.4333,0.4326,0.4327,438691.2,42.299473,-0.000147,2.449976e+06,-14.827202,0.001118,14.472921,67.498071,-0.001154,1.0,0.0
36,2022-10-01 09:00:00,0.4326,0.4335,0.4326,0.4329,716682.7,44.044221,-0.000156,3.166658e+06,2.053991,0.001102,13.950194,71.388210,0.000462,1.0,0.0
37,2022-10-01 09:15:00,0.4330,0.4336,0.4329,0.4330,600199.4,44.940696,-0.000153,3.766858e+06,27.000576,0.001073,13.572348,71.554002,0.000231,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,2023-01-02 11:30:00,0.2534,0.2539,0.2530,0.2536,1309851.5,56.550769,0.000792,-3.060913e+08,-109.157509,0.000947,43.791708,17.977815,0.000789,1.0,0.0
8924,2023-01-02 11:45:00,0.2537,0.2542,0.2537,0.2540,491283.2,59.454520,0.000755,-3.056000e+08,-29.778555,0.000922,43.191938,24.828020,0.001577,1.0,0.0
8925,2023-01-02 12:00:00,0.2541,0.2547,0.2541,0.2544,590081.8,62.176727,0.000750,-3.050099e+08,45.921986,0.000907,43.055906,32.111569,0.001575,0.0,0.0
8926,2023-01-02 12:15:00,0.2543,0.2547,0.2539,0.2544,485615.1,62.176727,0.000737,-3.050099e+08,33.685446,0.000899,42.548067,34.645950,0.000000,0.0,1.0


In [7]:
data = data.drop(['time','CLOSE_percent'],axis=1)
data

,open,high,low,close,volume,RSI,MACD,OBV,CCI,ATR,ADX,MFI,UP,DOWN
33,0.4337,0.4350,0.4334,0.4338,4464874.4,49.942739,-0.000119,3.504787e+06,145.526746,0.001131,15.868327,69.424814,0.0,1.0
34,0.4339,0.4342,0.4328,0.4332,616120.1,45.600201,-0.000113,2.888667e+06,72.832370,0.001150,15.261395,63.556057,0.0,1.0
35,0.4333,0.4333,0.4326,0.4327,438691.2,42.299473,-0.000147,2.449976e+06,-14.827202,0.001118,14.472921,67.498071,1.0,0.0
36,0.4326,0.4335,0.4326,0.4329,716682.7,44.044221,-0.000156,3.166658e+06,2.053991,0.001102,13.950194,71.388210,1.0,0.0
37,0.4330,0.4336,0.4329,0.4330,600199.4,44.940696,-0.000153,3.766858e+06,27.000576,0.001073,13.572348,71.554002,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,0.2534,0.2539,0.2530,0.2536,1309851.5,56.550769,0.000792,-3.060913e+08,-109.157509,0.000947,43.791708,17.977815,1.0,0.0
8924,0.2537,0.2542,0.2537,0.2540,491283.2,59.454520,0.000755,-3.056000e+08,-29.778555,0.000922,43.191938,24.828020,1.0,0.0
8925,0.2541,0.2547,0.2541,0.2544,590081.8,62.176727,0.000750,-3.050099e+08,45.921986,0.000907,43.055906,32.111569,0.0,0.0
8926,0.2543,0.2547,0.2539,0.2544,485615.1,62.176727,0.000737,-3.050099e+08,33.685446,0.000899,42.548067,34.645950,0.0,1.0


In [8]:
minmaxsc = MinMaxScaler()
data = minmaxsc.fit_transform(data)
data

array([[0.98273235, 0.9748996 , 0.98528666, ..., 0.689055  , 0.        ,
        1.        ],
       [0.9837481 , 0.97088353, 0.98224252, ..., 0.62937064, 0.        ,
        1.        ],
       [0.98070086, 0.96636546, 0.9812278 , ..., 0.66946032, 1.        ,
        0.        ],
       ...,
       [0.07059421, 0.06977912, 0.07559614, ..., 0.30958497, 0.        ,
        0.        ],
       [0.07160995, 0.06977912, 0.07458143, ..., 0.33535923, 0.        ,
        1.        ],
       [0.07160995, 0.0687751 , 0.07508879, ..., 0.29336799, 0.        ,
        0.        ]])

In [9]:

X,y = list(),list()
ref_bar = 10

for i in range(len(data)-ref_bar):
    #data.iloc.values 會回傳一個numpy array
    X.append(data[i:i+ref_bar, 0:12]) # i to i+ref_bar-1
    y.append(data[i+ref_bar-1, 12:]) # i+ref_bar-1



In [15]:
X = torch.tensor(X, dtype=torch.float32)

C:\Users\李軒豪\AppData\Local\Temp\ipykernel_36392\205850233.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


In [16]:
X[0]

tensor([[0.9827, 0.9749, 0.9853, 0.9832, 0.1400, 0.4600, 0.4332, 0.9670, 0.6770,
         0.0831, 0.1106, 0.6891],
        [0.9837, 0.9709, 0.9822, 0.9802, 0.0180, 0.4096, 0.4335, 0.9652, 0.5905,
         0.0852, 0.1019, 0.6294],
        [0.9807, 0.9664, 0.9812, 0.9777, 0.0124, 0.3712, 0.4319, 0.9639, 0.4862,
         0.0817, 0.0907, 0.6695],
        [0.9771, 0.9674, 0.9812, 0.9787, 0.0212, 0.3915, 0.4315, 0.9660, 0.5063,
         0.0801, 0.0832, 0.7090],
        [0.9792, 0.9679, 0.9827, 0.9792, 0.0175, 0.4019, 0.4316, 0.9677, 0.5360,
         0.0770, 0.0778, 0.7107],
        [0.9792, 0.9654, 0.9787, 0.9792, 0.0124, 0.4019, 0.4318, 0.9677, 0.4423,
         0.0765, 0.0687, 0.7035],
        [0.9792, 0.9689, 0.9833, 0.9807, 0.0112, 0.4357, 0.4332, 0.9689, 0.5639,
         0.0744, 0.0618, 0.7627],
        [0.9812, 0.9694, 0.9843, 0.9827, 0.0189, 0.4783, 0.4358, 0.9708, 0.6021,
         0.0718, 0.0569, 0.7699],
        [0.9827, 0.9699, 0.9802, 0.9812, 0.0110, 0.4461, 0.4367, 0.9696, 0.5225,

In [17]:
y[0]

array([0., 1.])

In [19]:
X.shape

torch.Size([8885, 10, 12])

In [23]:
total_cnt = 0
correct_cnt = 0
for index in range(len(X)):
    output = model(X[index])
    total_cnt += 1
    if output[0] > output[1] and y[index][0] == 1:
        correct_cnt += 1
    elif output[0] < output[1] and y[index][1] == 1:
        correct_cnt += 1

print("Accuracy: ", correct_cnt/total_cnt)

Accuracy:  0.770793472144063
